In [1]:
import numpy as np
import tensorflow as tf
import edward as ed
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from edward.models import Normal, Bernoulli

import preprocess_data

In [2]:
file_path = './flavors_of_cacao.csv'

In [3]:
x_train, x_test, y_train, y_test = preprocess_data.preprocess_data(file_path, test_pct=0.2)

/Users/allenchang/PycharmProjects/edward_one/cacao/preprocess_data.py:53: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  np.logical_or(cacao['species'].str.lower().str.contains(',|(blend)|;'),


In [4]:
N = x_train.shape[0]  # Number of rows in training data
D = x_train.shape[1]  # Number of columns in training data
M = 32  # Mini-batch size

x_ph = tf.placeholder(tf.float32, [None, D])  # Placeholder variable for x data
y_ph = tf.placeholder(tf.float32, [None])  # Placeholder variable for y data 

w = Normal(loc=tf.zeros(D), scale=tf.ones(D))  # Weights prior
b = Normal(loc=tf.zeros(1.), scale=tf.ones(1.))  # Bias prior
y = Normal(loc=ed.dot(x_ph, w) + b, scale=1.0)  # Likelihood function

In [5]:
qw = Normal(loc=tf.get_variable('qw/loc', [D]),
            scale=tf.nn.softplus(tf.get_variable('qw/scale', [D])))  # Inferred parameter
qb = Normal(loc=tf.get_variable('qb/loc', [1]),
            scale=tf.nn.softplus(tf.get_variable('qb/scale', [1])))  # Inferred parameter

In [6]:
def generator(arrays, batch_size):
  """Generate batches, one with respect to each array's first axis."""
  starts = [0] * len(arrays)  # pointers to where we are in iteration
  while True:
    batches = []
    for i, array in enumerate(arrays):
      start = starts[i]
      stop = start + batch_size
      diff = stop - array.shape[0]
      if diff <= 0:
        batch = array[start:stop]
        starts[i] += batch_size
      else:
        batch = np.concatenate((array[start:], array[:diff]))
        starts[i] = diff
      batches.append(batch)
    yield batches

data = generator([x_train, y_train], M)

In [7]:
n_batch = int(N / M)
n_epoch = 5

inference = ed.KLqp({w: qw, b: qb}, data={y: y_ph})
inference.initialize(
    n_iter=n_batch * n_epoch, n_samples=10, scale={y: N / M}, logdir='log')
tf.global_variables_initializer().run()

In [8]:
for _ in range(inference.n_iter):
    X_batch, y_batch = next(data)
    info_dict = inference.update({x_ph:X_batch, y_ph: y_batch})
    inference.print_progress(info_dict)

220/220 [100%] ██████████████████████████████ Elapsed: 4s | Loss: 3622.374


In [9]:
y_post = ed.copy(y, {w: qw, b: qb})

In [10]:
print("Mean squared error on test data:")
print(ed.evaluate('mean_squared_error', data={x_ph: x_test, y_post: y_test}))

print("Mean absolute error on test data:")
print(ed.evaluate('mean_absolute_error', data={x_ph: x_test, y_post: y_test}))

Mean squared error on test data:


/Users/allenchang/PycharmProjects/edward_one/venv/lib/python3.6/site-packages/edward/util/random_variables.py:52: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  not np.issubdtype(value.dtype, np.float) and \


1.8846741
Mean absolute error on test data:
1.1492201
